In [1]:
import os

import pandas as pd
from openai import AzureOpenAI

In [ ]:
client = AzureOpenAI(
  api_key = "",  
  api_version = "",
  azure_endpoint = ""
)

In [3]:
response = client.chat.completions.create(
    model = "gpt-4-1106-preview",
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Who were the founders of Microsoft?"}
    ]
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

{
  "id": "chatcmpl-A1g6hhkvu3oRdOx6GuULX81z0MYJW",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Microsoft was founded by Bill Gates and Paul Allen on April 4, 1975. Gates and Allen had been friends since their teenage years and shared a common interest in computer programming. They started Microsoft to develop and sell BASIC interpreters for the Altair 8800, an early personal computer. The company grew to dominate the home computer operating system market with MS-DOS in the mid-1980s, followed by Microsoft Windows. Their partnership and vision were instrumental in propelling the growth of the personal computing industry.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "

In [4]:
SYSTEM_PROMPT = """You are a language model that translates x-ray reports from Spanish to English. You will always return a json.\
You will be provided a report and some localized label data that was extracted from the report. \
The labels were extracted using an old NLP model and are not 100\% accurate\n
You respond to the user with a json of the following format: {"output": "translated text"}\n
Do not add any additional information to the output. The output needs to be a direct translation of the input text."""
BASE_USER_PROMPT = "Please translate the following x-ray report into English: "

In [5]:
def call_openai(REPORT="torax suci . valor con clinic pacient .", EXTRA=""):
    USER_PROMPT = BASE_USER_PROMPT + REPORT + EXTRA
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT}
        ]
    )
    output = response.choices[0].message.content
    completion_tokes = response.usage.completion_tokens
    prompt_tokens = response.usage.prompt_tokens
    
    return output, completion_tokes, prompt_tokens

In [14]:
prompt = "  import cambi cronic parenquim pulmon asoci probabl bronquiectasi infrahiliar bilateral . hili prominent orig vascul . ateromatosis aortic calcific . marcapas unicameral proyect region pectoral izquierd . cerclaj esternotomi medi ." + "\n" + \
                "Localized Label Data: [['aortic atheromatosis', 'loc aortic'], ['bronchiectasis', 'chronic changes', 'loc bronchi', 'loc infrahilar', 'loc bilateral'], ['pacemaker', 'single chamber device', 'loc pectoral', 'loc left'], ['sternotomy'], ['vascular hilar enlargement', 'loc hilar']]"
x = call_openai(REPORT=prompt)

In [16]:
import json
json.loads(x[0])['output']

'Chronic parenchymal pulmonary changes associated with probable bilateral infrahilar bronchiectasis. Hilar enlargement of vascular origin. Aortic atheromatosis with calcification. A unicameral pacemaker projects over the left pectoral region. Circumferential sternotomy wire.'

In [71]:
x

('{"output": "Chronic changes in the pulmonary parenchyma associated with probable bilateral bronchiectasis in the infrahilar region. Prominent hilum of vascular origin. Calcified aortic atheromatosis. Single-chamber pacemaker projected over the left pectoral region. Sternal cerclage from median sternotomy."}',
 71,
 221)

In [3]:
padchest_path = "../data/PadChest/PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv"
padchest = pd.read_csv(padchest_path)
padchest.head(3)

,Unnamed: 0,ImageID,ImageDir,StudyDate_DICOM,StudyID,PatientID,PatientBirth,PatientSex_DICOM,ViewPosition_DICOM,Projection,...,ExposureTime,RelativeXRayExposure_DICOM,ReportID,Report,MethodLabel,Labels,Localizations,LabelsLocalizationsBySentence,labelCUIS,LocalizationsCUIS
0,0,20536686640136348236148679891455886468_k6ga29.png,0,20140915,20536686640136348236148679891455886468,839860488694292331637988235681460987,1930.0,F,POSTEROANTERIOR,PA,...,10.0,-1.42,4765777,sin hallazg patolog edad pacient .,Physician,['normal'],[],"[['normal'], ['normal']]",[],[]
1,1,135803415504923515076821959678074435083_fzis7d...,0,20150914,135803415504923515076821959678074435083,313572750430997347502932654319389875966,1929.0,M,LATERAL,L,...,25.0,NaN,4991845,cambi pulmonar cronic sever . sign fibrosis b...,Physician,"['pulmonary fibrosis', 'chronic changes', 'kyp...","['loc basal', 'loc basal bilateral']","[['pulmonary fibrosis', 'loc basal bilateral']...",['C0034069' 'C0742362' 'C2115817' 'C3544344'],['C1282378']
2,2,135803415504923515076821959678074435083_fzis7b...,0,20150914,135803415504923515076821959678074435083,313572750430997347502932654319389875966,1929.0,M,POSTEROANTERIOR,PA,...,10.0,NaN,4991845,cambi pulmonar cronic sever . sign fibrosis b...,Physician,"['pulmonary fibrosis', 'chronic changes', 'kyp...","['loc basal', 'loc basal bilateral']","[['pulmonary fibrosis', 'loc basal bilateral']...",['C0034069' 'C0742362' 'C2115817' 'C3544344'],['C1282378']


In [4]:
# Drop cases where Report is empty
print(len(padchest))
padchest = padchest.dropna(subset=["Report"])
print(len(padchest))

160861
160745


In [8]:
x = set(padchest.Report.to_list())

# count tokens in these unique reports
toks = [len(i.split()) for i in x]
print(sum(toks)/len(toks))
print(sum(toks))


20.473904566134127
1354574


In [19]:
# Deduplicate a list

def fast_deduplicate(lst):
    def to_hashable(item):
        if isinstance(item, list):
            return tuple(to_hashable(i) for i in item)
        return item

    seen = set()
    result = []
    for item in lst:
        hashable_item = to_hashable(item)
        if hashable_item not in seen:
            seen.add(hashable_item)
            result.append(item)
    return result

fast_deduplicate(eval(padchest.LabelsLocalizationsBySentence.to_list()[3]))

[['chronic changes'],
 ['normal', 'loc cardiac', 'loc mediastinum'],
 ['normal'],
 ['normal', 'loc costophrenic angle']]

In [20]:
print(padchest[padchest.ImageID == "12752243479320243012014_01-165-029.png"].Report.iloc[0])
padchest[padchest.ImageID == "12752243479320243012014_01-165-029.png"]

 comp con radiografi previ 10 juni desaparicion imag nodul adyacent cisur menor exist cambi inflamatori bas derech lm . engros pleural apical bilateral . con tract fibros


,Unnamed: 0,ImageID,ImageDir,StudyDate_DICOM,StudyID,PatientID,PatientBirth,PatientSex_DICOM,ViewPosition_DICOM,Projection,...,ExposureTime,RelativeXRayExposure_DICOM,ReportID,Report,MethodLabel,Labels,Localizations,LabelsLocalizationsBySentence,labelCUIS,LocalizationsCUIS
100978,100978,12752243479320243012014_01-165-029.png,31,20140729,12752243479320243012014,250700714735340203745291685829785100849,1943.0,M,POSTEROANTERIOR,PA,...,10.0,-0.89,4711887,comp con radiografi previ 10 juni desaparicio...,RNN_model,"['', 'apical pleural thickening', 'fibrotic ba...","['loc right', 'loc pleural', 'loc apical', 'lo...","['', 'loc right', 'loc basal', 'loc minor fiss...",['' 'C0865843'],['C0444532' 'C0032225' 'C0734296' 'C0458078' '...


In [26]:
# Clean up the padchest data


padchest_clean = padchest.copy()[['ImageID', 'StudyID', 'PatientID', \
    'PatientBirth','PatientSex_DICOM', 'Report', 'Labels', 'LabelsLocalizationsBySentence']]


def fix_labels(row):
    try:
        Labels = eval(row.Labels)
        Labels = fast_deduplicate(Labels)
        LabelsLocalizationsBySentence = eval(row.LabelsLocalizationsBySentence)
        LabelsLocalizationsBySentence = fast_deduplicate(LabelsLocalizationsBySentence)
        
        if "nan" in LabelsLocalizationsBySentence:
            LabelsLocalizationsBySentence = []
        if "nan" in Labels:
            Labels = []
    except:
        LabelsLocalizationsBySentence = []
        Labels = []
    
    row['LabelsLocalizationsBySentence'] = LabelsLocalizationsBySentence
    row['Labels'] = ",".join(Labels)
    return row

# padchest_clean.sample(10).apply(fix_labels, axis=1)
padchest_clean = padchest_clean.apply(fix_labels, axis=1)



In [27]:
padchest_clean.to_csv("../data/PadChest/padchest_clean.csv", index=False, sep="\t")

In [28]:
padchest_clean

,ImageID,StudyID,PatientID,PatientBirth,PatientSex_DICOM,Report,Labels,LabelsLocalizationsBySentence
0,20536686640136348236148679891455886468_k6ga29.png,20536686640136348236148679891455886468,839860488694292331637988235681460987,1930.0,F,sin hallazg patolog edad pacient .,normal,[[normal]]
1,135803415504923515076821959678074435083_fzis7d...,135803415504923515076821959678074435083,313572750430997347502932654319389875966,1929.0,M,cambi pulmonar cronic sever . sign fibrosis b...,"pulmonary fibrosis,chronic changes,kyphosis,ps...","[[pulmonary fibrosis, loc basal bilateral], [c..."
2,135803415504923515076821959678074435083_fzis7b...,135803415504923515076821959678074435083,313572750430997347502932654319389875966,1929.0,M,cambi pulmonar cronic sever . sign fibrosis b...,"pulmonary fibrosis,chronic changes,kyphosis,ps...","[[pulmonary fibrosis, loc basal bilateral], [c..."
3,113855343774216031107737439268243531979_3k951l...,113855343774216031107737439268243531979,50783093527901818115346441867348318648,1925.0,F,. . siluet cardi mediastin dentr normal . cam...,chronic changes,"[[chronic changes], [normal, loc cardiac, loc ..."
4,113855343774216031107737439268243531979_3k951n...,113855343774216031107737439268243531979,50783093527901818115346441867348318648,1925.0,F,. . siluet cardi mediastin dentr normal . cam...,chronic changes,"[[chronic changes], [normal, loc cardiac, loc ..."
...,...,...,...,...,...,...,...,...
160856,1284011361929414522814654121696751542351444145...,1284011361929414522814654121696751542351444145...,112930952416074060371371014599496493673,1948.0,M,import sign radiolog epoc . pinzamient ambos ...,"COPD signs,costophrenic angle blunting","[COPD signs, costophrenic angle blunting, loc ..."
160857,1284011361929414522094646571696751542351444145...,1284011361929414522094646571696751542351444145...,282743729971423358706056731890510600934,1944.0,F,dentr normal .,normal,[normal]
160858,1284011361929414522086390631696751542351444145...,1284011361929414522086390631696751542351444145...,52648743308541843883453242716226652771,1965.0,M,tub endotraqueal sond nasogastr situacion cor...,"NSG tube, endotracheal tube","[NSG tube, endotracheal tube, loc tracheal, n..."
160859,1284011361929414522084108901696751542351444145...,1284011361929414522084108901696751542351444145...,228646130593152933811948996634154201216,1943.0,F,dentr normal .,normal,[normal]
